In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
from datetime import datetime, timedelta, date

In [2]:
df_data = pd.read_csv('data.csv')

In [3]:
conn = sql.connect("db_SQL.db") 

In [7]:
df_data.to_sql('input_data', conn)

129201

In [8]:
## I am not completely sure that I've understood the expectation. Still, I used approach to query relevant attributes 
## from dataset needed to analyse overdue agreements. Create tabe if not exists is not supporther in Jupyter, however
## it is possible to test scripts here and ensure that they work well. 

query_b ='''

--create table if not exists {{reports}}.overdue_agreements AS is not supported in Jypiter, but can be used in other softwehere

with facts as (
     select loan_id,
            client_id
     from input_data
       where loan_issued = 1 --only issued agreements are needed for overdue analytics
)
select ft.loan_id,
       ft.client_id,
       dt.loan_term,
       date(dt.loan_created_on)                  loan_created_on, 
       date(dt.loan_paid_on)                     loan_paid_on,
       Cast ((JulianDay(dt.loan_paid_on) 
            - JulianDay(dt.loan_created_on)
            -  dt.loan_term
                  ) As Integer)                  overdue_in_days,
       dt.client_debt,
       date(dt.client_birth_date)                client_birth_date
from facts ft
    left join input_data dt on ft.loan_id = dt.loan_id
 where dt.loan_paid_on is not null
'''

In [9]:
df_task_1_b = pd.read_sql(query_b, conn)

In [10]:
df_task_1_b

,loan_id,client_id,loan_term,loan_created_on,loan_paid_on,overdue_in_days,client_debt,client_birth_date
0,1,1,69,2019-06-15,2019-08-23,0,0,1981-12-31
1,2,2,90,2019-05-23,2019-08-21,0,837,1998-09-10
2,6,6,30,2019-05-29,2019-06-28,0,50,1973-01-30
3,8,8,30,2019-05-24,2019-06-23,0,0,1986-11-13
4,9,9,90,2019-06-03,2019-09-01,0,12135,1993-10-17
...,...,...,...,...,...,...,...,...
62846,129183,75925,29,2021-04-06,2021-05-05,0,50,1976-11-01
62847,129186,75965,29,2021-04-06,2021-05-05,0,7861,2003-06-25
62848,129195,76366,19,2021-04-06,2021-04-25,0,0,2003-06-22
62849,129196,76470,28,2021-04-06,2021-05-04,0,0,1992-08-15


In [11]:
## Validates yearly and monthly dynamics of client’s characteristics who have not repaid the loans and 
## compares them to the ones who made repayment. 

query_c ='''
--create view if not exists {{reports}}.undapid_loan_client_data AS 
-- is not supported in Jypiter, but can be used in other softwehere

with base as (
     select loan_id
     from input_data
       where loan_issued = 1 --only issued agreements are needed for overdue analytics
       ),
    loan_repayed as (
       select strftime('%m-%Y', dt.loan_created_on)   creadted_on_mm_yyyy,
              dt.loan_id,
              dt.client_income,
              dt.client_expenses,
              (dt.client_expenses)/
                  (dt.client_income)                  client_expenses_to_income,
              dt.client_liabilities,
              dt.client_debt
       from base bs
             left join input_data dt on bs.loan_id = dt.loan_id 
        where dt.loan_paid_on is not null
        ),
   not_repayed_loan as (
       select strftime('%m-%Y', dt.loan_created_on)   creadted_on_mm_yyyy,
              dt.loan_id,
              dt.client_income,
              dt.client_expenses,
              (dt.client_expenses)/
                  (dt.client_income)                 client_expenses_to_income,
              dt.client_liabilities,
              dt.client_debt
       from base bs
             left join input_data dt on bs.loan_id = dt.loan_id 
        where dt.loan_paid_on is null
       ),
    total as (
           select coalesce(rp.creadted_on_mm_yyyy, nr.creadted_on_mm_yyyy)      creadted_on_mm_yyyy,
                  bs.loan_id,
                  coalesce (rp.client_income, nr.client_income)                            client_income,
                  coalesce (rp.client_expenses, nr.client_expenses)                        client_expenses,
                  coalesce (rp.client_expenses_to_income, nr.client_expenses_to_income)    client_expenses_to_income,
                  coalesce (rp.client_liabilities, nr.client_liabilities)                  client_liabilities,
                  coalesce (rp.client_debt, nr.client_debt)                                client_debt,
                  case 
                      when rp.creadted_on_mm_yyyy  is not null 
                          then  'repayed'
                          else 'not_repayed'  end                                          repayment_status
           from base bs
               left join loan_repayed rp on bs.loan_id = rp.loan_id     -- since full outer join not supported in sqlight
               left join not_repayed_loan nr on bs.loan_id = nr.loan_id
)
select creadted_on_mm_yyyy,
       repayment_status,
       avg(client_income)                avg_client_income,
       avg(client_expenses)              avg_client_expenses,
       avg(client_expenses_to_income)    avg_client_expenses_to_income,
       avg(client_liabilities)           avg_client_liabilities,
       avg(client_debt)                  avg_client_debt
from total 
group by 1,2
'''

In [12]:
df_task_1_c = pd.read_sql(query_c, conn)

In [13]:
df_task_1_c

,creadted_on_mm_yyyy,repayment_status,avg_client_income,avg_client_expenses,avg_client_expenses_to_income,avg_client_liabilities,avg_client_debt
0,01-2020,not_repayed,2540.924205,601.762836,0.224939,268.026895,4372.085575
1,01-2020,repayed,3075.254313,662.659989,0.124652,182.227045,1678.831942
2,01-2021,not_repayed,2384.038679,569.570755,0.183962,242.047170,3375.342453
3,01-2021,repayed,3145.469760,644.422764,0.106881,168.580607,2344.666865
4,02-2020,not_repayed,2491.333333,530.971545,0.154472,292.804878,2873.373984
5,02-2020,repayed,3085.330838,647.144461,0.097305,166.447605,2413.049401
6,02-2021,not_repayed,2822.242510,599.672696,0.195591,200.522894,3280.624647
7,02-2021,repayed,3217.266192,631.420533,0.115526,159.700735,2058.099449
8,03-2020,not_repayed,2357.947883,635.612378,0.325733,311.687296,3466.980456
9,03-2020,repayed,3153.007756,650.577839,0.122992,166.402770,1956.226593
